In [13]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [14]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE

application_df.drop(columns=['EIN', 'NAME'], inplace=True)

Optimization attempt: dropping the USE_CASE column from the dataframe. The category of the business venture may not be as important as other monetary factors in the data.

In [15]:
application_df.drop(columns=['USE_CASE'])

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,1,0,N,5000,1
1,T3,Independent,C2000,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Association,1,0,N,5000,0
34297,T5,Independent,C3000,Association,1,0,N,5000,1


In [16]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
unique_counts = application_df.nunique()

# Display the number of unique values in each column
print(unique_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [17]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE

application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the value counts
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [18]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

application_types_to_replace = application_type_counts[application_type_counts < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [19]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE

classification_counts = application_df['CLASSIFICATION'].value_counts()

# Display the value counts
print(classification_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [20]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#  YOUR CODE GOES HERE

classification_counts_gt_1 = classification_counts[classification_counts > 1]

# Display the filtered value counts
print(classification_counts_gt_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64


Optimization attempt: Changing the cutoff value from 1000 to 100 for classifications grouped as "Other". 100 organizations in a classification may be enough observations to draw conclusions about the group.

In [21]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE

classifications_to_replace = classification_counts[classification_counts < 100].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [22]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES

application_df_encoded = pd.get_dummies(application_df)

# Display the first few rows of the encoded DataFrame
print(application_df_encoded.head())

   STATUS  ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0       1     5000              1                       0   
1       1   108590              1                       0   
2       1     5000              0                       0   
3       1     6692              1                       0   
4       1   142590              1                       0   

   APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                     1                     0                    0   
1                     0                     0                    1   
2                     0                     0                    0   
3                     0                     0                    1   
4                     0                     0                    1   

   APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                    0                    0                    0  ...   
1                    0                    0                    0  

In [23]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

# Features (X): All columns except 'IS_SUCCESSFUL'
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL'])

# Target (y): 'IS_SUCCESSFUL' column
y = application_df_encoded['IS_SUCCESSFUL']


# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Optimization attempt: Decreasing number of hidden nodes to avoid overfitting

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 6
hidden_nodes_layer2 = 3


nn = tf.keras.models.Sequential()

# First hidden layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
#  YOUR CODE GOES HERE

nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 6)                 300       
                                                                 
 dense_7 (Dense)             (None, 3)                 21        
                                                                 
 dense_8 (Dense)             (None, 1)                 4         
                                                                 
Total params: 325 (1.27 KB)
Trainable params: 325 (1.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Compile the model
#  YOUR CODE GOES HERE

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [35]:
# Train the model
#  YOUR CODE GOES HERE

from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint callback
checkpoint_path = "model_checkpoint.h5"
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path,
                                      save_weights_only=True,
                                      save_freq='epoch',
                                      period=5)

# Train the model with the defined callback
history = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
858/858 [==============================] - 7s 6ms/step - loss: 0.6298 - accuracy: 0.6837
Epoch 2/50
858/858 [==============================] - 2s 3ms/step - loss: 0.5793 - accuracy: 0.7207
Epoch 3/50
858/858 [==============================] - 2s 2ms/step - loss: 0.5707 - accuracy: 0.7227
Epoch 4/50
858/858 [==============================] - 2s 3ms/step - loss: 0.5646 - accuracy: 0.7239
Epoch 5/50
858/858 [==============================] - 2s 3ms/step - loss: 0.5591 - accuracy: 0.7283
Epoch 6/50
858/858 [==============================] - 3s 4ms/step - loss: 0.5568 - accuracy: 0.7282
Epoch 7/50
858/858 [==============================] - 3s 4ms/step - loss: 0.5558 - accuracy: 0.7290
Epoch 8/50
858/858 [==============================] - 4s 4ms/step - loss: 0.5552 - accuracy: 0.7285
Epoch 9/50
858/858 [==============================] - 3s 4ms/step - loss: 0.5545 - accuracy: 0.7291
Epoch 10/50
858/858 [==============================] - 4s 5ms/step - loss: 0.5541 - accuracy: 0.7296

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5559 - accuracy: 0.7274 - 750ms/epoch - 3ms/step
Loss: 0.555853009223938, Accuracy: 0.7274052500724792


In [37]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE

model_file = "AlphabetSoupcharity_Optimization.h5"

# Save the model to HDF5 format
nn.save(model_file)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
